Run this cell to initialize population synthesis

In [ ]:
import sys
import os
from types import SimpleNamespace
sys.path.append(os.path.abspath(".."))

from synthesize import *
from preprocess import *

test_on = "lyon"

if test_on == "lyon":
    config_folder = "../configs/Lyon"
    ploc = None
elif test_on == "chicago":
    config_folder = "../configs/Chicago"
    ploc = puma_locations(config_folder)

model_conf, synth_conf, survey_conf, analysis = load_config(config_folder=config_folder)

# namespace for synth configuration
synth = SimpleNamespace(**synth_conf)
n = synth.sample_size
subsample = synth.subsample
batch_size = synth.batch_size
source = synth.source
shuffle_response = synth.shuffle_response
shuffle_prompt = synth.shuffle_response
wrap = synth.wrap
read_from_dataset = synth.read_from_dataset

# person = process_pums_data(config_folder)
if test_on == "lyon":
    person = process_insee_census(config_folder=config_folder)
else:
    person = process_pums_data(config_folder = config_folder)

population_sample = synthesize_population(
    config_folder=config_folder, n_sample=5000, min_age=18, max_age=65, random_state=2,
    source=source,
    read_from_dataset=read_from_dataset
)

year = 2019

output = []
attribute_descriptions = get_attribute_descriptions(person)

In [ ]:
from collections import Counter

smg = SystemMessageGenerator(
    config_folder,
    "SystemMessage.j2",
    verbose_debug=False,
    shuffle=False,
    wrap=80
    )

msg = []
for i, individual in population_sample.reset_index().iterrows():
    individual_attributes = attribute_decoder_dict(individual.to_dict(), person)
    system_message = smg.write_system_message(**individual_attributes, **attribute_descriptions, ploc=ploc, YEAR=year)
    print(system_message+"\n")

In [ ]:
var = "MOCO"

counts = population_sample[var].value_counts()
print(counts)

responses = person.get(var)
responses

In [ ]:
var = "STATR"

counts = population_sample[var].value_counts()
print(counts)

responses = person.get(var)
responses

In [ ]:
population_sample.groupby("TACT")["STATR"].value_counts()